In [1]:
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

from pathlib import Path, PurePath
import pandas as pd
import os

In [2]:
with open("method_metrics_non_zero.txt", "r") as f:
    method_metrics_list = f.readlines()
    method_metrics_list = [x.strip() for x in method_metrics_list]

In [3]:
with open("class_metrics_non_zero.txt", "r") as f:
    class_metrics_list = f.readlines()
    class_metrics_list = [x.strip() for x in class_metrics_list]

In [5]:
df_test = pd.read_csv(f"GA_Sammon_method.csv").set_index("Repository")

new_row = {str(x): x for x in range(2, 21)}
df_test

,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Repository,,,,,,,,,,,,,,,,,,,
aio-libs/aiohttp,524304,528400,37392,20816,1578128,1118739,292948,1055293,954907,1588948,1030929,2087760,1900369,1048404,1957687,2072287,2080734,2097111,2097119
aleju/imgaug,2064,524312,663568,331793,268340,264982,661785,532273,815921,163508,1660567,760758,1744699,1039934,1023807,966623,2072319,2096887,2097119
ansible/ansible,524304,540688,794640,528913,21525,6996,817232,1711312,589329,540404,898716,1014482,1023801,1810237,2079702,1023967,2031485,2097111,2031615
apache/tvm,262160,4120,1058832,12624,792080,531484,547985,1708722,889497,801459,2004824,2088466,1940155,1998682,654975,2023135,2096855,2031551,2097119
AUTOMATIC1111/stable-diffusion-webui,4112,528400,534544,557140,276504,1578516,1063452,628376,1334806,1480219,1851185,2029201,1867697,2088601,1048245,2088631,2072567,2097111,2096895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
open-mmlab/mmdetection,2064,1050640,786452,1578000,300050,302097,823325,318994,1858194,548509,916020,1015322,2006772,1957683,917175,2080599,2096951,2080735,2097119
python-poetry/poetry,4112,557072,5392,135441,1577010,725552,1318034,832021,401043,1621904,695923,1023573,1924956,2072245,1032178,1941439,1949439,2080511,2097119
ray-project/ray,262160,524306,266384,786453,692248,284945,1841936,1578396,1875218,1865237,1005520,2039449,1982165,1761179,1998495,1031927,2096951,2072575,2097119


In [6]:
df_test.at['aio-libs/aiohttp', '2']

524304

In [8]:
len(metric_list_non_zero)

NameError: name 'metric_list_non_zero' is not defined

In [9]:
(pd.read_csv(f"{method}_{error}_{metric_type}.csv").set_index("Repository") == 0).sum().sum()

NameError: name 'method' is not defined

In [18]:
method = "PSO"
error = "Sammon"
metric_type = "method"
extra_num = 1

df = pd.read_csv(f"{method}_{error}_{metric_type}.csv").set_index("Repository")

if metric_type == "class":
    metric_list_non_zero = np.array(class_metrics_list)
elif metric_type == "method":
    metric_list_non_zero = np.array(method_metrics_list)
else:
    raise Exception()
    
#repo_list = os.listdir(f'Results/{method}_{error}/{metric_type}')
repo_list = list(Path("Results").glob(f'{method}_{error}*/{metric_type}/*'))
repo_list = [str(x).split('\\')[-1] for x in repo_list]
repo_list = [x for x in repo_list if x[-5:] == '.json']
repo_list_cleaned = [x[:-5].replace('_', '/') for x in repo_list]

repos_len = len(repo_list)
metrics_len = len(metric_list_non_zero)

out = np.zeros(shape=(len(repo_list_cleaned), metrics_len-2), dtype=np.int64)

for file_idx, file_name in enumerate(repo_list):
    if not os.path.isfile(f'Results/{method}_{error}_{metric_type}_Extra_{extra_num}/{metric_type}/{file_name}'):
        continue
        
    with open(f'Results/{method}_{error}_{metric_type}_Extra_{extra_num}/{metric_type}/{file_name}', 'r') as f:
        file_data = f.readline()
        file_data = json.loads(file_data)
        
    for idx, n_metrics in enumerate(range(2, len(metric_list_non_zero))):
        
        if file_name.replace('_', '/')[:-5] not in df.index:
            new_row = {str(x): 0 for x in range(2, 21)}
            df = df.append(pd.Series(new_row, index=df.columns, name=file_name.replace('_', '/')[:-5]))
        
        
        #print(df.at[file_name.replace('_', '/')[:-5], str(n_metrics)][0])
        if df.at[file_name.replace('_', '/')[:-5], str(n_metrics)] != 0:
            continue
        
        if str(n_metrics) not in file_data:
            continue
        best_n_metrics = file_data[str(n_metrics)]['feat_idx']
        best_n_metrics = metric_list_non_zero[best_n_metrics]
        
        bitmask = 0
        for metric in best_n_metrics:
            bitmask = bitmask | 2**np.where(metric_list_non_zero == metric)[0]
        
        df.at[file_name.replace('_', '/')[:-5], str(n_metrics)] = bitmask

            


df_out = df.copy()
df_out = df_out.reset_index()
df_out.to_csv(f"{method}_{error}_{metric_type}.csv", index=False)
print(f"Number of NaN's: {(df_out == 0).sum().sum()}")

Number of NaN's: 0


In [19]:
df_out

,Repository,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,aio-libs/aiohttp,4112,528400,528912,791056,793104,794128,826896,1842708,1875476,1875664,1875605,2072148,2072149,2072277,2014935,2080471,2080479,2097111,2097119
1,aleju/imgaug,524304,524816,528912,791056,531984,924176,925200,794133,925205,1973782,1973783,1973911,2039383,2072151,2072471,2088663,2088671,2088927,2097119
2,ansible/ansible,4112,528400,528912,530960,793104,794128,1841684,1842708,1973780,826965,2006549,2006551,2006741,2072151,2072279,2080471,2080727,2097111,2097119
3,apache/tvm,4112,528400,528912,530960,793104,924176,925200,925201,925206,957973,1973783,1973911,2006615,2072151,2072407,2072535,2080727,2097111,2097119
4,AUTOMATIC1111/stable-diffusion-webui,4112,528400,528912,530960,793104,1580560,1842704,1842708,826901,1908245,1875541,2006613,1941079,2072151,2080343,2080471,2080727,2097111,2097119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,open-mmlab/mmdetection,524304,528400,530448,530960,531984,794128,925200,925201,925206,1973781,1973783,2006551,2072087,2072151,2072279,2072535,2080727,2097111,2097119
102,python-poetry/poetry,4112,528400,530448,530960,531984,531988,794132,1842708,1875476,1843028,2006612,2006613,2006741,2072277,2072279,2080471,2080727,2097111,2097119
103,ray-project/ray,4112,6160,530448,530960,793104,924176,925200,925201,925205,1973782,1973783,2006551,2006615,2006743,2072279,2072535,2080727,2097111,2097119
104,sqlmapproject/sqlmap,4112,4624,528912,530960,530964,793108,794132,925204,1973780,1973781,2006549,2072085,2072149,2072277,2072279,2072535,2088919,2097111,2097119


In [20]:
(pd.read_csv(f"GA_Sammon_method.csv").set_index("Repository") == 0).sum().sum()

0

In [21]:
(pd.read_csv(f"GA_Sammon_class.csv").set_index("Repository") == 0).sum().sum()

0

In [22]:
(pd.read_csv(f"PSO_Sammon_method.csv").set_index("Repository") == 0).sum().sum()

0

In [23]:
(pd.read_csv(f"PSO_Sammon_class.csv").set_index("Repository") == 0).sum().sum()

0

In [24]:
pd.read_csv(f"GA_Sammon_method.csv").set_index("Repository")

,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Repository,,,,,,,,,,,,,,,,,,,
aio-libs/aiohttp,524304,528400,37392,20816,1578128,1118739,292948,1055293,954907,1588948,1030929,2087760,1900369,1048404,1957687,2072287,2080734,2097111,2097119
aleju/imgaug,2064,524312,663568,331793,268340,264982,661785,532273,815921,163508,1660567,760758,1744699,1039934,1023807,966623,2072319,2096887,2097119
ansible/ansible,524304,540688,794640,528913,21525,6996,817232,1711312,589329,540404,898716,1014482,1023801,1810237,2079702,1023967,2031485,2097111,2031615
apache/tvm,262160,4120,1058832,12624,792080,531484,547985,1708722,889497,801459,2004824,2088466,1940155,1998682,654975,2023135,2096855,2031551,2097119
AUTOMATIC1111/stable-diffusion-webui,4112,528400,534544,557140,276504,1578516,1063452,628376,1334806,1480219,1851185,2029201,1867697,2088601,1048245,2088631,2072567,2097111,2096895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
open-mmlab/mmdetection,2064,1050640,786452,1578000,300050,302097,823325,318994,1858194,548509,916020,1015322,2006772,1957683,917175,2080599,2096951,2080735,2097119
python-poetry/poetry,4112,557072,5392,135441,1577010,725552,1318034,832021,401043,1621904,695923,1023573,1924956,2072245,1032178,1941439,1949439,2080511,2097119
ray-project/ray,262160,524306,266384,786453,692248,284945,1841936,1578396,1875218,1865237,1005520,2039449,1982165,1761179,1998495,1031927,2096951,2072575,2097119


In [25]:
pd.read_csv(f"PSO_Sammon_method.csv").set_index("Repository")

,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Repository,,,,,,,,,,,,,,,,,,,
aio-libs/aiohttp,4112,528400,528912,791056,793104,794128,826896,1842708,1875476,1875664,1875605,2072148,2072149,2072277,2014935,2080471,2080479,2097111,2097119
aleju/imgaug,524304,524816,528912,791056,531984,924176,925200,794133,925205,1973782,1973783,1973911,2039383,2072151,2072471,2088663,2088671,2088927,2097119
ansible/ansible,4112,528400,528912,530960,793104,794128,1841684,1842708,1973780,826965,2006549,2006551,2006741,2072151,2072279,2080471,2080727,2097111,2097119
apache/tvm,4112,528400,528912,530960,793104,924176,925200,925201,925206,957973,1973783,1973911,2006615,2072151,2072407,2072535,2080727,2097111,2097119
AUTOMATIC1111/stable-diffusion-webui,4112,528400,528912,530960,793104,1580560,1842704,1842708,826901,1908245,1875541,2006613,1941079,2072151,2080343,2080471,2080727,2097111,2097119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
open-mmlab/mmdetection,524304,528400,530448,530960,531984,794128,925200,925201,925206,1973781,1973783,2006551,2072087,2072151,2072279,2072535,2080727,2097111,2097119
python-poetry/poetry,4112,528400,530448,530960,531984,531988,794132,1842708,1875476,1843028,2006612,2006613,2006741,2072277,2072279,2080471,2080727,2097111,2097119
ray-project/ray,4112,6160,530448,530960,793104,924176,925200,925201,925205,1973782,1973783,2006551,2006615,2006743,2072279,2072535,2080727,2097111,2097119


In [70]:
bin(37392)

'0b1001001000010000'

In [71]:
bin(528400)

'0b10000001000000010000'

In [150]:
df = pd.read_csv(f"PSO_Sammon_class.csv").set_index("Repository")
print(f"Number of NaN's: {(df == 0).sum().sum()}")

Number of NaN's: 0


In [124]:
df.loc['huggingface/transformers']

2       98304
3      327712
4       67620
5      327730
6      594032
7      246064
8     1675301
9      651305
10     364090
11          0
12    1543223
13     783155
14     767295
15    1303999
16          0
17     915199
18    2092798
19    2092991
20    2097147
Name: huggingface/transformers, dtype: int64

In [144]:
cols = [str(x) for x in range(2, 21)]
a = pd.DataFrame(columns=cols)
a.index.name = "Repository"
a

,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Repository,,,,,,,,,,,,,,,,,,,
